In [98]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [99]:
# change the path to where your notebook is located
%cd "gdrive/My Drive/Colab Notebooks/NLI course"

[Errno 2] No such file or directory: 'gdrive/My Drive/Colab Notebooks/NLI course'
/home/adrian/facultate/emai/NLI/NLI_project


In [100]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


In [101]:
# Run it only once
from datasets import load_dataset
dataset = load_dataset("multi_woz_v22")

No config specified, defaulting to: multi_woz_v22/v2.2_active_only
Found cached dataset multi_woz_v22 (/home/adrian/.cache/huggingface/datasets/multi_woz_v22/v2.2_active_only/2.2.0/6719c8b21478299411a0c6fdb7137c3ebab2e6425129af831687fb7851c69eb5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [102]:
# Run it only once
dataset.save_to_disk("dataset.hf")

Saving the dataset (0/1 shards):   0%|          | 0/8437 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [103]:
from datasets import DatasetDict
dataset = DatasetDict.load_from_disk("dataset.hf")

In [104]:
dataset['train'].features

{'dialogue_id': Value(dtype='string', id=None),
 'services': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'turns': Sequence(feature={'turn_id': Value(dtype='string', id=None), 'speaker': ClassLabel(names=['USER', 'SYSTEM'], id=None), 'utterance': Value(dtype='string', id=None), 'frames': Sequence(feature={'service': Value(dtype='string', id=None), 'state': {'active_intent': Value(dtype='string', id=None), 'requested_slots': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'slots_values': Sequence(feature={'slots_values_name': Value(dtype='string', id=None), 'slots_values_list': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}, length=-1, id=None)}, 'slots': Sequence(feature={'slot': Value(dtype='string', id=None), 'value': Value(dtype='string', id=None), 'start': Value(dtype='int32', id=None), 'exclusive_end': Value(dtype='int32', id=None), 'copy_from': Value(dtype='string', id=None), 'copy_from_value': Sequence(featur

In [105]:
dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [106]:
dataset['train'][0].keys()

dict_keys(['dialogue_id', 'services', 'turns'])

In [107]:
print(len(dataset["train"]), len(dataset["validation"]), len(dataset["test"]))

8437 1000 1000


In [108]:
set([service for dial in dataset['train'] for service in dial['services']])

{'attraction', 'bus', 'hospital', 'hotel', 'restaurant', 'taxi', 'train'}

In [109]:
set([service for dial in dataset['train'] for turn_id,utt in enumerate(dial['turns']['utterance']) for service in dial['turns']['frames'][turn_id]['service']])

{'attraction',
 'bus',
 'hospital',
 'hotel',
 'police',
 'restaurant',
 'taxi',
 'train'}

In [110]:
dataset['train'][112]

{'dialogue_id': 'SSNG0297.json',
 'services': ['hotel'],
 'turns': {'turn_id': ['0',
   '1',
   '2',
   '3',
   '4',
   '5',
   '6',
   '7',
   '8',
   '9',
   '10',
   '11',
   '12',
   '13'],
  'speaker': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
  'utterance': ['Can you find me a 3 star hotel in the centre?',
   'The Gonville Hotel meets your requirements. Would you like me to book you a room?',
   'Does it have free wifi? I need wifi.',
   'Yes, they do offer free internet. Would you like me to make a booking?',
   'Yes. I want to book it for 1 person. I want to stay for 5 nights starting from Wednesday.',
   'I am sorry Booking was unsuccessful,can I help you find another place to stay?',
   'Yes please. Another 3-star hotel in the centre with free wifi.',
   'I do not have another 3 star hotel in the centre that meets your requirements. Would you like to try another area?',
   'Do you have any hotels in the same price range as the Gonville Hotel?',
   'The only other hotel that

In [111]:
def Dialogue_Act_Prediction(utterance, other_features_from_dialogue_history):
    return ['Hotel-Request']

def Extract_and_Categorize_Spans(utterance, dialogue_acts, other_features_from_dialogue_history2):
    return [('hotel-bookpeople', '2'), ('hotel-bookstay', '2'), ('hotel-bookday', 'sunday'), ('restaurant-phone', '?')]

def Agent_Move_Prediction(dialogue_acts, extracted_information, other_features_from_dialogue_history3):
    return {"dialogue_acts": ["Booking-Request", "Hotel-Recommend", "Restaurant-Inform"],
            "retrieved_information": {"to_be_provided": set(['hotel-area', 'hotel-name', 'restaurant-name', 'restaurant-phone', 'restaurant-address', 'booking-ref']),
                                      "to_be_requested": set(['booking-bookday', 'booking-bookpeople', 'restaurant-pricerange'])}}

In [112]:
def print_output(*output):
    if do_print_dialogue_details:
        if len(output) == 1:
            print(output[0])
        elif len(output) == 2:
            print(output[0], output[1])
        elif len(output) == 3:
            print(output[0], output[1], output[2])
        elif len(output) == 4:
            print(output[0], output[1], output[2], output[3])
        else:
            print(output)

def count_matches(ground_truth_list, predicted_list):
    no_gt = len(ground_truth_list)
    no_predicted = len(predicted_list)
    no_correct = no_gt - sum((Counter(ground_truth_list) - Counter(predicted_list)).values())
    return no_gt, no_predicted, no_correct

def get_metrics(no_gt_global, no_predicted_global, no_correct_global):
    precision = 1.0*no_correct_global/no_predicted_global
    recall = 1.0*no_correct_global/no_gt_global
    f1_score = 2.0*precision*recall/(precision+recall)
    return precision, recall, f1_score

In [114]:
from collections import Counter

global do_print_dialogue_details
do_print_dialogue_details = True
print_ground_truth_structures = True
print_predicted = False

no_moves_gt_global = 0 # "gt" == "ground truth"
no_moves_predicted_global = 0
no_moves_correct_global = 0

n_dialogues_to_evaluate = 10
n_evaluated = 0
for d,dial in enumerate(dataset['train']):
    if n_evaluated>=n_dialogues_to_evaluate:
        break
    # skip dialogues that are not in the hotel or restaurant domain
    if not any(set(dial['turns']['frames'][turn_id]['service']).intersection(['hotel', 'restaurant']) for turn_id,utt in enumerate(dial['turns']['utterance'])):
        continue
    #if not any('Hotel-Request' in dial['turns']['dialogue_acts'][turn_id]['dialog_act']['act_type'] and dial['turns']['speaker'][turn_id]==0 for turn_id,utt in enumerate(dial['turns']['utterance'])):
    #    continue
    #if not any('Hotel-Request' in dial['turns']['dialogue_acts'][turn_id]['dialog_act']['act_type'] and dial['turns']['speaker'][turn_id]==1 for turn_id,utt in enumerate(dial['turns']['utterance'])):
    #    continue
    print_output("Dialogue ID:", dial['dialogue_id'])
    compulsory_slots_hotel  = set(['hotel-bookpeople', 'hotel-bookstay', 'hotel-name', 'hotel-bookday']) # as an example, to be adjusted
    compulsory_slots_restaurant  = set(['restaurant-name']) # as an example, to be adjusted
    filled_slots = set()
    speaker_str = {0: 'User', 1: 'Agent'}
    turns = dial['turns']
    for turn_id,utt in enumerate(turns['utterance']):
        speaker = speaker_str[turns['speaker'][turn_id]]
        if speaker == "User":
            print_output("User's turn")
            print_output("User's utterance: "+utt)

            print_output("Extraction")

            indent = " "*4
            dialogue_acts = turns['dialogue_acts'][turn_id]['dialog_act']['act_type']
            dialogue_acts_ground_truth = dialogue_acts.copy()
            print_output(indent, "Dialogue acts:", dialogue_acts)

            dialogue_acts_predicted = Dialogue_Act_Prediction(utt, other_features_from_dialogue_history={})
            if print_ground_truth_structures:
                print(dialogue_acts_ground_truth)

            # evaluate user's dialogue acts
            no_gt, no_predicted, no_correct = count_matches(dialogue_acts_ground_truth, dialogue_acts_predicted)
            # add global variables here as in "evaluate agent's dialogue acts"
            if print_predicted:
                print(indent, "User's dialogue acts predicted:", dialogue_acts_predicted, "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

            print_output(indent, "Extracted information:")
            print_output(indent, "Spans")
            extracted_information_not_mapped_ground_truth = []
            extracted_information_ground_truth = []
            extracted_information_per_dialogue_act_ground_truth = {}
            span_info = turns['dialogue_acts'][turn_id]['span_info']
            for span_i in range(len(span_info['span_start'])):
                act_type = span_info['act_type'][span_i]
                span_name = span_info['act_slot_name'][span_i]
                span_value = span_info['act_slot_value'][span_i]
                span_range = [span_info['span_start'][span_i], span_info['span_end'][span_i]]
                span_value_as_in_utterance = utt[span_info['span_start'][span_i]: span_info['span_end'][span_i]]
                print_output(indent*2, span_value + ("" if span_value_as_in_utterance==span_value else " ("+span_value_as_in_utterance+")"), span_range)
                if not act_type in extracted_information_per_dialogue_act_ground_truth:
                    extracted_information_per_dialogue_act_ground_truth[act_type] = []
                extracted_information_not_mapped_ground_truth.append(tuple([act_type.split("-")[0].lower()+"-"+span_name, span_value_as_in_utterance]))
                extracted_information_ground_truth.append(tuple([act_type.split("-")[0].lower()+"-"+span_name, span_value]))
                extracted_information_per_dialogue_act_ground_truth[act_type].append(tuple([span_name, span_value]))

            print_output(indent, "Categorized information")
            slot_names_per_act = [slot['slot_name'] for slot in turns['dialogue_acts'][turn_id]['dialog_act']['act_slots']]
            slot_values_per_act = [slot['slot_value'] for slot in turns['dialogue_acts'][turn_id]['dialog_act']['act_slots']]
            for act_i in range(len(slot_names_per_act)):
                slot_names_values_per_act = [slot_names_per_act[act_i][slot_i]+":"+slot_values_per_act[act_i][slot_i] for slot_i in range(len(slot_names_per_act[act_i]))]
                print_output(indent*2, dialogue_acts[act_i], slot_names_values_per_act)
                if dialogue_acts[act_i].startswith("Hotel") or dialogue_acts[act_i].startswith("Restaurant"):
                    for slot_i in range(len(slot_names_per_act[act_i])):
                        if slot_names_per_act[act_i][slot_i] != "none" and slot_values_per_act[act_i][slot_i] == "?":
                            if not dialogue_acts[act_i] in extracted_information_per_dialogue_act_ground_truth:
                                extracted_information_per_dialogue_act_ground_truth[dialogue_acts[act_i]] = []
                            extracted_information_not_mapped_ground_truth.append(tuple([dialogue_acts[act_i].split("-")[0].lower()+"-"+slot_names_per_act[act_i][slot_i], slot_values_per_act[act_i][slot_i]]))
                            extracted_information_ground_truth.append(tuple([dialogue_acts[act_i].split("-")[0].lower()+"-"+slot_names_per_act[act_i][slot_i], slot_values_per_act[act_i][slot_i]]))
                            extracted_information_per_dialogue_act_ground_truth[dialogue_acts[act_i]].append(tuple([slot_names_per_act[act_i][slot_i], slot_values_per_act[act_i][slot_i]]))

            extracted_information = Extract_and_Categorize_Spans(utt, dialogue_acts_ground_truth, other_features_from_dialogue_history2={})

            if print_ground_truth_structures:
                print("OK")
                print(extracted_information_not_mapped_ground_truth)
                print(extracted_information_ground_truth)
                print(extracted_information_per_dialogue_act_ground_truth)
                print("NOK")

            # evaluate information extraction only if there are Hotel or Restaurant or general dialogue acts
            no_gt, no_predicted, no_correct = count_matches(extracted_information_ground_truth, extracted_information)
            if any(da.startswith("general") or da.startswith("Hotel") or da.startswith("Restaurant") for da in dialogue_acts_ground_truth):
                # add global variables here as in "evaluate agent's dialogue acts"
                if print_predicted:
                    print(indent, "Extracted and categorized information (predicted):", extracted_information, "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

            no_gt, no_predicted, no_correct = count_matches(agent_move_ground_truth["retrieved_information"]["to_be_provided"], agent_move_predicted["retrieved_information"]["to_be_provided"])

            print_output("Reasoning")
            services = turns['frames'][turn_id]['service']
            print_output(indent, "Services:", services)

            not_empty_intents = [intent for intent in turns['frames'][turn_id]['state'] if intent['requested_slots'] or intent['slots_values']['slots_values_name']]
            if not_empty_intents:
                print_output(indent, "Intents")
                for intent in not_empty_intents:
                    print_output(indent*2, "Active intent:", intent['active_intent'])
                    requested_slots = intent['requested_slots']
                    if requested_slots:
                        print_output(indent*2, "Requested slots:", requested_slots)
                    if intent['slots_values']['slots_values_name']:
                        slot_names = intent['slots_values']['slots_values_name']
                        slot_values = intent['slots_values']['slots_values_list']
                        filled_slots.update(slot_names)
                        print_output(indent*2, "Filled slots:")
                        for slot_i in range(len(slot_names)):
                            print_output(indent*3, slot_names[slot_i]+": ", slot_values[slot_i])
                    print_output(indent*2, "--------------")

            print_output(indent, "Missing slots (Hotel):", compulsory_slots_hotel - filled_slots)
            print_output(indent, "Missing slots (Restaurant):", compulsory_slots_restaurant - filled_slots)
        elif speaker == "Agent":
            print_output("Agent's turn")

            print_output("Planning")

            indent = " "*4
            dialogue_acts = turns['dialogue_acts'][turn_id]['dialog_act']['act_type']
            print_output(indent, "Agent's move (dialogue acts):", dialogue_acts)

            print_output("Retrieval")

            slot_names_per_act = [slot['slot_name'] for slot in turns['dialogue_acts'][turn_id]['dialog_act']['act_slots']]
            slot_values_per_act = [slot['slot_value'] for slot in turns['dialogue_acts'][turn_id]['dialog_act']['act_slots']]

            print_output(indent, "Retrieved information:")
            for act_i in range(len(slot_names_per_act)):
                print_output(indent*2, dialogue_acts[act_i])
                print_output(indent*2, "To be provided:", [slot_names_per_act[act_i][slot_i]+":"+slot_values_per_act[act_i][slot_i] for slot_i in range(len(slot_names_per_act[act_i])) if slot_values_per_act[act_i][slot_i]!="?"])
                print_output(indent*2, "To be requested:", [slot_names_per_act[act_i][slot_i]+":"+slot_values_per_act[act_i][slot_i] for slot_i in range(len(slot_names_per_act[act_i])) if slot_values_per_act[act_i][slot_i]=="?"])
                print_output(indent*2, "--------------")

            agent_move_ground_truth = {
                "dialogue_acts": dialogue_acts,
                "retrieved_information": {"to_be_provided": set(),  # only unique names of the slots
                                          "to_be_requested": set()  # only unique names of the slots
                                          },
                "retrieved_information_per_dialogue_act": {} # non-none names of the slots and values grouped per dialogue act
                }

            for act_i in range(len(slot_names_per_act)):
                if dialogue_acts[act_i].startswith("Hotel") or dialogue_acts[act_i].startswith("Restaurant") or dialogue_acts[act_i].startswith("Booking"):
                    for slot_i in range(len(slot_names_per_act[act_i])):
                        if slot_names_per_act[act_i][slot_i] != "none":
                            if not dialogue_acts[act_i] in agent_move_ground_truth["retrieved_information_per_dialogue_act"]:
                                agent_move_ground_truth["retrieved_information_per_dialogue_act"][dialogue_acts[act_i]] = []
                            agent_move_ground_truth["retrieved_information_per_dialogue_act"][dialogue_acts[act_i]].append(tuple([slot_names_per_act[act_i][slot_i], slot_values_per_act[act_i][slot_i]]))
                            if slot_values_per_act[act_i][slot_i]!="?":
                                agent_move_ground_truth["retrieved_information"]["to_be_provided"].add(dialogue_acts[act_i].split("-")[0].lower()+"-"+slot_names_per_act[act_i][slot_i])
                            else:
                                agent_move_ground_truth["retrieved_information"]["to_be_requested"].add(dialogue_acts[act_i].split("-")[0].lower()+"-"+slot_names_per_act[act_i][slot_i])

            agent_move_predicted = Agent_Move_Prediction(dialogue_acts, extracted_information_per_dialogue_act_ground_truth, other_features_from_dialogue_history3={})
            if print_ground_truth_structures:
                print(agent_move_ground_truth)

            if print_predicted:
                print("Planning predicted")

            # evaluate agent's dialogue acts
            no_gt, no_predicted, no_correct = count_matches(agent_move_ground_truth["dialogue_acts"], agent_move_predicted["dialogue_acts"])
            no_moves_gt_global += no_gt
            no_moves_predicted_global += no_predicted
            no_moves_correct_global += no_correct
            if print_predicted:
                print(indent, "Agent's dialogue acts predicted:", agent_move_predicted["dialogue_acts"], "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

            # evaluate "retrieved_information" -> "to_be_provided" only if there are Hotel or Restaurant or Booking or general dialogue acts
            no_gt, no_predicted, no_correct = count_matches(agent_move_ground_truth["retrieved_information"]["to_be_provided"], agent_move_predicted["retrieved_information"]["to_be_provided"])
            if any(da.startswith("general") or da.startswith("Hotel") or da.startswith("Restaurant") or da.startswith("Booking") for da in agent_move_ground_truth["dialogue_acts"]):
                # add global variables here as in "evaluate agent's dialogue acts"
                if print_predicted:
                    print(indent, "Info to be provided predicted:", agent_move_predicted["retrieved_information"]["to_be_provided"], "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

            # evaluate "retrieved_information" -> "to_be_requested" only if there are Hotel or Restaurant or Booking or general dialogue acts
            no_gt, no_predicted, no_correct = count_matches(agent_move_ground_truth["retrieved_information"]["to_be_requested"], agent_move_predicted["retrieved_information"]["to_be_requested"])
            if any(da.startswith("general") or da.startswith("Hotel") or da.startswith("Restaurant") or da.startswith("Booking") for da in agent_move_ground_truth["dialogue_acts"]):
                # add global variables here as in "evaluate agent's dialogue acts"
                if print_predicted:
                    print(indent, "Info to be requested predicted:", agent_move_predicted["retrieved_information"]["to_be_requested"], "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

            print_output("Agent's utterance: "+utt)
        print_output("-------------------------------------------------------------------")
        print_output("-------------------------------------------------------------------")
    n_evaluated+=1

print("Dialogue acts in the agent's move prediction")
precision, recall, f1_score = get_metrics(no_moves_gt_global, no_moves_predicted_global, no_moves_correct_global)
print("Precision: %lf, Recall: %lf, F1-score: %lf" % (precision, recall, f1_score))

Dialogue ID: PMUL4398.json
User's turn
User's utterance: i need a place to dine in the center thats expensive
Extraction
     Dialogue acts: ['Restaurant-Inform']
['Restaurant-Inform']
     Extracted information:
     Spans
         centre (center) [30, 36]
         expensive [43, 52]
     Categorized information
OKKK
         Restaurant-Inform ['area:centre', 'pricerange:expensive']
OK
[('restaurant-area', 'center'), ('restaurant-pricerange', 'expensive')]
[('restaurant-area', 'centre'), ('restaurant-pricerange', 'expensive')]
{'Restaurant-Inform': [('area', 'centre'), ('pricerange', 'expensive')]}
NOK


NameError: name 'agent_move_ground_truth' is not defined